# Tello Tempo - D

## Import de bibliothèques

In [ ]:
!pip install tellopy
!pip install djitellopy

In [ ]:
from djitellopy import Tello
import tellopy
import cv2 as cv 
import socket, time
from time import sleep
import datetime
import os
from threading import Thread

import pytz # timezone for timestamp
import datetime


### Useful custom functions

In [ ]:
def get_timestamp():
    tz = pytz.timezone('Europe/Paris')
    currentDT = datetime.datetime.now(tz)
    TIMESTAMP = currentDT.strftime("%Y%m%d_%H%M%S")
    return TIMESTAMP

In [ ]:
def battery_level(tello):
    print(f"Battery level is {tello.get_battery()}%")

In [ ]:
def difference(list1, list2):
    zip_object = zip(list1, list2)
    difference=[]
    for list1_i, list2_i in zip_object:
        difference.append(list2_i-list1_i)
    return difference

# Utilisation  de `Tellopy`

* Décollage et attérissage 

In [ ]:

# Paramètres de communication
host = ''
port = 9000
locaddr = (host,port)

#Connexion à l'adresse IP du drône pour entamer la communication 
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
tello_address = ('192.168.10.1', 8889)
sock.bind(locaddr)


sock.sendto('command'.encode(encoding="utf-8"), tello_address)
time.sleep(5)

# # Takeoff
# sock.sendto('takeoff'.encode(encoding="utf-8"), tello_address)
time.sleep(5)


#Rotate clockwise 360
# sock.sendto('tcw 360'.encode(encoding="utf-8"), tello_address)
time.sleep(5)

# Land
# sock.sendto('land'.encode(encoding="utf-8"), tello_address)
time.sleep(5)

# Utilisation de  `Djitellopy`

##  Décollage , photo et attérissage 

In [ ]:
tello= Tello () 
tello.connect() # Connexion
tello.takeoff() # Décollage 
# tello.streamon() # Activation de la caméra

time.sleep(1) # Tempo de 2s 
frame_read = tello.get_frame_read() # Capture d'impage
img_name = f"view_{get_timestamp()}.png"
print(img_name)
cv.imwrite("testnew.png", frame_read.frame) # Stockage en local 
time.sleep(1)
tello.streamoff()
tello.land()   # Attérissage

*  Capture de vidéo

In [ ]:
tello = Tello()
tello.connect()

KeepRecording= True
tello.streamon()

frame_read = tello.get_frame_read()

def videoRecorder():
    height, width,_= frame_read.frame.shape
    video_name = f"video_{get_timestamp()}.avi"

    video = cv.VideoWriter(video_name, cv.VideoWriter_fourcc(*'XVID'), 30, (width,height))
    
    while KeepRecording:
        video.write(frame_read.frame)
        time.sleep(1/30)
    video.release()
    
recorder  = Thread(target=videoRecorder)
recorder.start()

tello.takeoff()
tello.land()
KeepRecording= False
recorder.join()



## Patrol - Tello.curve

In [ ]:
def patrol():
    """
    Dessine un arc de cercle et se retourne
    Espae nécessaire environ 1.5m
    """
    tello.curve_xyz_speed(25, -25, 0, 150, 25, 0, 50)
    tello.rotate_clockwise(180)


In [ ]:
NUM_PATROL = 4

tello = Tello()
tello.connect(False)

# afficher niveau de batterie
battery_level(tello)

tello.takeoff()
tello.move_up(80)

for k in range(NUM_PATROL):
    patrol()
    
tello.land()

In [ ]:
tello??

### Tentative de vol spirale

* TODO donner de bonnes coordonnées
* TODO fluidité des changement de coordonnées

In [ ]:
def move_spirale(t, pos):
    return [np.cos(t)*10, np.sin(t)*10, pos[2]+t, 0]

In [ ]:
def spirale(tello):
    TEMPO=0.5
    t=0
    pos=[20, 20, 0, 0]
    speed=50
    PADDING=10

    next_pos = pos
    for k in range(10):
        t+=1
        pos_ = move_spirale(t, next_pos)
        next_pos = difference(next_pos, pos_)
        if next_pos[0] > 0:
            next_pos[0]+=PADDING
        else:
            next_pos[0]-=PADDING
        if next_pos[1] > 0:
            next_pos[1]+=PADDING
        else:
            next_pos[1]-=PADDING
        print(t, next_pos)
        tello.go_xyz_speed(int(next_pos[0]), int(next_pos[1]), int(next_pos[2])+PADDING, speed)
        time.sleep(TEMPO)


In [ ]:
tello = Tello()
tello.connect(False)

# afficher niveau de batterie
battery_level(tello)

tello.takeoff()
# tello.move_up(80)

spirale(tello)
    
tello.land()